# Import libs

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor

import os, sys
sys.path.append("../..")

import warnings
warnings.filterwarnings("ignore")

# Import Data

#### define paths

In [5]:
path_hash_data = "../../data/HASH_and_target_time_series.parquet"
path_centrobank_data = "../../data/Kluch_stavka_and_Inflation.csv"
path_rub_to_usd_and_yuan_data = "../../data/kurs_rub_to_usd_and_uan.csv"
path_urals_in_usd_data = "../../data/urals_oil_price.csv"

#### load datasets

In [6]:
df_hash_data = pd.read_parquet(path_hash_data)
# df_centrobank_data = pd.read_csv(path_centrobank_data, sep=";")
df_curs_data = pd.read_csv(path_rub_to_usd_and_yuan_data)
df_urals_money_data = pd.read_csv(path_urals_in_usd_data)

In [7]:
df_curs_data.index = pd.to_datetime(df_curs_data.data, yearfirst=True)
df_curs_data = df_curs_data.drop(axis=1, columns=['data'])

In [8]:
df_urals_money_data.index = pd.to_datetime(df_urals_money_data.data, dayfirst=True)
df_urals_money_data = df_urals_money_data.drop(axis=1, columns=["data"])

In [10]:
df_additional_features = pd.merge(df_urals_money_data, df_curs_data, left_index=True, right_index=True)

In [11]:
df_additional_features = df_additional_features.rename(columns={"price": "price_urals_usd"})

In [12]:
df = pd.merge(df_hash_data, df_additional_features, left_index=True, right_index=True)